In [32]:
import serial

import time
import csv
import matplotlib
matplotlib.use("tkAgg")
import matplotlib.pyplot as plt
import numpy as np

import re
import pandas as pd
import dask.dataframe as dd
import streamz.dataframe as sdf
# from streamz.dataframe import StreamingDataFrame

# import threading, queue
import multiprocess as mp
# from multiprocessing import Process, Pipe
import signal
import os
from subprocess import check_output

from scipy.ndimage import gaussian_filter1d

In [9]:
portAddr = "/dev/cu.usbmodem14101"
baud = 115200

In [10]:
ser = serial.Serial(
    port=portAddr,
    baudrate=baud,
    bytesize=8,
    timeout=2,
    stopbits=serial.STOPBITS_ONE
)
ser.flushInput()



In [11]:
# plot_window = 20
# y_var = np.array(np.zeros([plot_window]))

# plt.ion()
# fig, ax = plt.subplots()
# line, = ax.plot(y_var)


In [12]:
def process_serial_out(ascii_string):
    ascii_string += ','
    findall = re.findall(r'(?P<var>.*?):(?P<out>.*?),', ascii_string)
    extracted = {k.strip(): v.strip() for k,v in findall}
    return extracted

In [13]:

def arduino_worker(queue):
    print("worker starting...")
    while True:
        try:
            ser_bytes = ser.readline()
            try:
                print("got", ser_bytes)
                decoded_bytes = ser_bytes.decode('ascii')
                row = process_serial_out(decoded_bytes)
                print(decoded_bytes)
                queue.put(row)
            except:
                print_exc()
                print("exception reading one line from arduino")
                continue
        except:
            print_exc()
            print("Keyboard Interrupt")
            break

In [14]:
import random

def stride_boundary(row, this_stride):
#     just a dummy code. We need to implement the logic.
    if (bool(random.getrandbits(1))):
        return True
    else:
        return False

In [34]:
class Stride:
    def __init__(self, df_walk_data, start_idx, stop_idx):
        self.start_idx = start_idx
        self.stop_idx = stop_idx
        self.df_walk_data = df_walk_data

        total_disp, integral1, integral2 = get_displacement_of_walk_segment (df_walk_data, start_idx, stop_idx)
        self.horizontal_length = abs(total_disp)
        self.time_duration = (df_walk_data.time[self.stop_idx] - df_walk_data.time[self.start_idx])/1000
        self.speed = self.horizontal_length/self.time_duration #m/s


def get_displacement_of_walk_segment(df_data, start_idx, end_idx):
    # first integration
    integral_0 = df_data.accel_z[start_idx:end_idx]

    # first integral
    cumsum = 0
    integral_1 = []
    for idx, elem in enumerate(integral_0):
        delta_time = (df_data.time[start_idx+idx+1] - df_data.time[start_idx+idx] ) / 1000
        cumsum += elem * delta_time
        integral_1.append(cumsum)
    
    # second integral
    cumsum = 0
    integral_2 = []
    for idx, elem in enumerate(integral_1):
        delta_time = (df_data.time[start_idx+idx+1] - df_data.time[start_idx+idx] ) / 1000
        cumsum += elem * delta_time
        integral_2.append(cumsum)
    
    # total_displacement, first_integral, second_integral
    return cumsum, integral_1, integral_2

segment_length = 100
temp_segment_compute_limit_start = 0
temp_segment_compute_limit_end = 400
# temp_segment_compute_limit_end = len(readings)
total_walk = []
strides = []

# Algorithm Constants
start_quiet_noise_idx = 0
end_quiet_noise_idx = 100 # 1 second sigma calibration
STRIDE_TIMEOUT = 500
num_of_stddev = 3
gaussian_filter_sigma = 5


In [ ]:
def process_calculate_strides(total_data):
    print("starting calculate_strides")
    while True:
        for segment_start in np.arange(0,len(readings),segment_length):
            if (segment_start >= temp_segment_compute_limit_start and
                segment_start <= temp_segment_compute_limit_end): # TODO: Remove this IF statment at the end

                walk_segment = readings[segment_start:segment_start+segment_length]
                for k in walk_segment: total_walk.append(k) 
                df_walk = pd.DataFrame(total_walk)
                df_walk.time = df_walk.time - df_walk.time[0] # reset time from start of stream

                # Filter out accel_z
                accel_z_smooth = gaussian_filter1d(df_walk.accel_z, gaussian_filter_sigma)

                # Define Baseline Noise

                accel_z_std = np.std(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx], axis=0)
                accel_z_mean = np.mean(df_walk.accel_z[start_quiet_noise_idx:end_quiet_noise_idx])
                upper_limit = accel_z_mean + num_of_stddev * accel_z_std
                lower_limit = accel_z_mean - num_of_stddev * accel_z_std

                # Determining Stride
                # Stride Class with auto-compute all chracterisitcs
                stride_under = False
                stride_over = False
                start_stride_idx, end_stride_idx = None, None

                for idx, data in enumerate(accel_z_smooth):
                    # timeout
                    if start_stride_idx and idx-start_stride_idx > STRIDE_TIMEOUT:
                        start_stride_idx = None
                        stride_under = False
                        stride_over = False
                    if (data < lower_limit) and (not stride_under and not stride_over):
                        # Heal off the ground
                        stride_under = True
                        start_stride_idx = idx
                    elif (data > upper_limit) and (stride_under and not stride_over):
                        stride_under = False
                        stride_over = True
                    elif (data < upper_limit) and (not stride_under and stride_over):
                        stride_under = False
                        stride_over = False   
                        end_stride_idx = idx
                        curr_stride = Stride (df_walk, start_stride_idx, end_stride_idx)
                        strides.append(curr_stride)

In [ ]:


#worker to process one stride boundary, sort data into buckets
def stride_processor_worker(total_data, raw_stream):
    this_segment = []
    print("stride_processor starting...")
    while True:
        #Infinitely 1. appends to this_segment, then when boundary is met,
        #appends that stride to total_data, reinitializes this_segment
        #and loops forever.
        try:
            row = raw_stream.get()
            print ("decoded: ", row)
            if len(this_segment) > segment_length:
                #put away the current stride in total data as one stride
                total_data.put(this_segment)
                print("finished writing this one stride, size: ", len(this_segment))
                #reinitialize the stride queue
                this_segment = []
            else:
                this_segment.append(row)
        except:
            print_exc()
            print("exception in stride processor worker")
            
    

In [16]:
# overall structure:
# total_data = [stride1, stride2,,,,,,stride500]
# stride1 = [{r1}, {r2}, {r3}]

In [17]:
######### start 'main' ########

In [18]:
pool = mp.Pool()
manager = mp.Manager()
usb_reading_queue = mp.SimpleQueue()
total_data = mp.SimpleQueue()

In [19]:
#  set up worker threads

In [20]:
stride_process = mp.Process(target=stride_processor_worker, args=[total_data, usb_reading_queue])


In [31]:
arduino_proc = mp.Process(target=arduino_worker, args=[usb_reading_queue])


In [22]:
arduino_proc.start()

worker starting...
got b',Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491612,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491612,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491619,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491619,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491626,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y

got b'Time:5491775,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491775,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491782,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491782,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491790,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491790,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -


got b'Time:5491943,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491943,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491950,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491950,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5491957,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5491957,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: 


got b'Time:5492112,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492112,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492120,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492120,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492128,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492128,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: 


got b'Time:5492278,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492278,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492286,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492286,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492293,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492293,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: 


got b'Time:5492449,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492449,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492456,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492456,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05

got b'Time:5492464,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: -0.05,Mag_x: 0.72,Magl_y: 0.69,Mag_z: -0.05\r\n'
Time:5492464,Pitch:5.94,Yaw:88.17,Roll:-177.63,Accel_x: 0.72,Accel_y: 0.69,Accel_z: -0.05,Gyro_x: 0.72,Gyro_y: 0.69,Gyro_z: 

In [23]:
stride_process.start()

stride_processor starting...
decoded:  {',Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491612', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491619', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491626', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491632', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gy

decoded:  {'Time': '5491853', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491861', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491867', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491873', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5491881', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63'

decoded:  {'Time': '5492092', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
finished writing this one stride, size:  6
decoded:  {'Time': '5492100', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492106', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492112', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492120', 'Pitch'

decoded:  {'Time': '5492329', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492336', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
finished writing this one stride, size:  6
decoded:  {'Time': '5492343', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492351', 'Pitch': '5.94', 'Yaw': '88.17', 'Roll': '-177.63', 'Accel_x': '0.72', 'Accel_y': '0.69', 'Accel_z': '-0.05', 'Gyro_x': '0.72', 'Gyro_y': '0.69', 'Gyro_z': '-0.05', 'Mag_x': '0.72', 'Magl_y': '0.69'}
decoded:  {'Time': '5492358', 'Pitch'

In [24]:
# arduino_proc.join()
# stride_process.join()
arduino_proc.terminate()
stride_process.terminate()

In [25]:
total_data

In [26]:
total_data.empty()

False

In [29]:
stride_1 = total_data.get()

In [30]:
stride_1

[{',Gyro_x': '0.72',
  'Gyro_y': '0.69',
  'Gyro_z': '-0.05',
  'Mag_x': '0.72',
  'Magl_y': '0.69'},
 {'Time': '5491612',
  'Pitch': '5.94',
  'Yaw': '88.17',
  'Roll': '-177.63',
  'Accel_x': '0.72',
  'Accel_y': '0.69',
  'Accel_z': '-0.05',
  'Gyro_x': '0.72',
  'Gyro_y': '0.69',
  'Gyro_z': '-0.05',
  'Mag_x': '0.72',
  'Magl_y': '0.69'},
 {'Time': '5491619',
  'Pitch': '5.94',
  'Yaw': '88.17',
  'Roll': '-177.63',
  'Accel_x': '0.72',
  'Accel_y': '0.69',
  'Accel_z': '-0.05',
  'Gyro_x': '0.72',
  'Gyro_y': '0.69',
  'Gyro_z': '-0.05',
  'Mag_x': '0.72',
  'Magl_y': '0.69'},
 {'Time': '5491626',
  'Pitch': '5.94',
  'Yaw': '88.17',
  'Roll': '-177.63',
  'Accel_x': '0.72',
  'Accel_y': '0.69',
  'Accel_z': '-0.05',
  'Gyro_x': '0.72',
  'Gyro_y': '0.69',
  'Gyro_z': '-0.05',
  'Mag_x': '0.72',
  'Magl_y': '0.69'},
 {'Time': '5491632',
  'Pitch': '5.94',
  'Yaw': '88.17',
  'Roll': '-177.63',
  'Accel_x': '0.72',
  'Accel_y': '0.69',
  'Accel_z': '-0.05',
  'Gyro_x': '0.72',
  '